# Objectives

In this final part you will:

Run the ETL process

Extract bank and market cap data from the JSON file bank_market_cap.json

Transform the market cap currency using the exchange rate data

Load the transformed data into a seperate CSV

For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. 
The cells below will install these libraries when executed.

In [1]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

# Imports
Import any additional libraries you may need here.

In [2]:
import glob
import pandas as pd
from datetime import datetime

        As the exchange rate fluctuates, we will download the same dataset to make marking simpler.

In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-03-17 14:39:51--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: 'bank_market_cap_1.json'

     0K ..                                                    100% 1.38G=0s

2023-03-17 14:39:51 (1.38 GB/s) - 'bank_market_cap_1.json' saved [2815/2815]

--2023-03-17 14:39:51--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transfo

In [60]:
columns=['Name','Market Cap (US$ Billion)']

In [61]:
df1 = pd.read_json('bank_market_cap_1.json')
df2 = pd.read_json('bank_market_cap_2.json')
df = pd.concat([df1, df2])
print(df.head())

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399


# Extract
JSON Extract Function


This function will extract JSON files.

In [11]:
import json

def extract_json(filename):
    with open(filename) as f:
        data = json.load(f)
    return data
data1 = extract_json('bank_market_cap_1.json')
data2 = extract_json('bank_market_cap_2.json')

# Extract Function

Define the extract function that finds JSON file bank_market_cap_1.json and calls the function created above to extract data from them. 

Store the data in a pandas dataframe. Use the following list for the columns.

In [12]:
def extract():
    json_file = 'bank_market_cap_1.json'
    
    if os.path.exists(json_file):
        with open(json_file) as f:
            data = json.load(f)
        
        # Extract data from JSON file
        extracted_data = extract_json_data(data)
        
        # Store data in a pandas dataframe
        columns = ['Bank Name', 'Market Capitalization', 'Country', 'Headquarters']
        df = pd.DataFrame(extracted_data, columns=columns)
        
        return df
    else:
        print(f"{json_file} not found.")

In [46]:
import pandas as pd
import json
import logging

logging.basicConfig(level=logging.INFO)

def extract_data(data):
    extracted_data = []
    for company in data:
        extracted_data.append([company['Name'], company['Market Cap (US$ Billion)']])
    return extracted_data

def extract():
    file_path = 'bank_market_cap_1.json'
    with open(file_path) as f:
        data = json.load(f)
        extracted_data = extract_data(data['bank_market_cap_1'])
        df = pd.DataFrame(extracted_data, columns=['Name', 'Market Cap (US$ Billion)'])
        return df
    
def transform(data):
    transformed_data = data.copy()
    transformed_data['Market Cap (US$ Billion)'] = transformed_data['Market Cap (US$ Billion)'] / 1000
    return transformed_data

### Question 1 Load the file exchange_rates.csv as a dataframe and find the exchange rate for British pounds with the symbol GBP, 
store it in the variable exchange_rate, you will be asked for the number. Hint: set the parameter index_col to 0.

In [13]:
# Write your code here
# Load the CSV file into a dataframe
df = pd.read_csv('exchange_rates.csv', index_col=0)

# Find the exchange rate for GBP
exchange_rate = df.loc['GBP', 'Rates']

# Print the exchange rate for GBP
print("The exchange rate for GBP is:", exchange_rate)

The exchange rate for GBP is: 0.7323984208000001


# Transform

Using exchange_rate and the exchange_rates.csv file find the exchange rate of USD to GBP. Write a transform function that

Changes the Market Cap (US$ Billion) column from USD to GBP

Rounds the Market Cap (US$ Billion)` column to 3 decimal places

Rename Market Cap (US$ Billion) to Market Cap (GBP$ Billion)

In [14]:
def transform(df, exchange_rate):
    # Convert the Market Cap (US$ Billion) column to GBP
    df['Market Cap (GBP$ Billion)'] = df['Market Cap (US$ Billion)'] * exchange_rate
    
    # Round the Market Cap (GBP$ Billion) column to 3 decimal places
    df['Market Cap (GBP$ Billion)'] = df['Market Cap (GBP$ Billion)'].round(3)
    
    # Drop the Market Cap (US$ Billion) column
    df = df.drop(columns=['Market Cap (US$ Billion)'])
    
    # Rename the columns
    df = df.rename(columns={'Company Name': 'Company', 'Sector': 'Industry'})
    
    return df


In [15]:
# Load the original dataframe
df = pd.read_json('bank_market_cap_1.json')

# Find the exchange rate for USD to GBP
exchange_rates_df = pd.read_csv('exchange_rates.csv', index_col=0)
exchange_rate = exchange_rates_df.loc['GBP', 'Rates']

# Transform the dataframe
df_transformed = transform(df, exchange_rate)

# Print the transformed dataframe
print(df_transformed)

                                       Name  Market Cap (GBP$ Billion)
0                            JPMorgan Chase                    286.319
1   Industrial and Commercial Bank of China                    252.834
2                           Bank of America                    238.272
3                               Wells Fargo                    225.588
4                   China Construction Bank                    188.519
..                                      ...                        ...
65                             Ping An Bank                     27.826
66                       Standard Chartered                     27.332
67                     United Overseas Bank                     25.728
68                                QNB Group                     24.579
69                              Bank Rakyat                     24.228

[70 rows x 2 columns]


# Load

Create a function that takes a dataframe and load it to a csv named bank_market_cap_gbp.csv. Make sure to set index to False.

In [16]:
def load(df):
    data.to_csv('bank_market_cap.csv', index=False)

    # Write your code here

# Logging Function

Write the logging function log to log your dat

In [17]:
def log(data):
    logging.basicConfig(filename='my_log.log', level=logging.DEBUG)
    logging.info(f"Data: {data}")

# Running the ETL Process

Log the process accordingly using the following "ETL Job Started" and "Extract phase Started"

In [21]:
# Set up logging
import logging
logging.basicConfig(filename='etl.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Define extract function for JSON files
def extract_json(file_path):
    logging.info(f"Extracting data from {file_path}")
    # Code to extract data from JSON file goes here
    data = pd.read_json(file_path)
    return data

def extract_data():
    logging.info("Extract phase Started")
    # Call the extract_json function to extract data from JSON files
    data1 = extract_json('bank_market_cap_1.json')
    data2 = extract_json('bank_market_cap_2.json')
    # Combine the data into a single dataframe
    data = pd.concat([data1, data2])
    return data

# Define the list of columns for the dataframe
columns = ['bank_name', 'market_cap', 'country']

# Define the ETL process function
def run_etl():
    logging.info("ETL Job Started")
    # Call the extract_data function to extract the data
    data = extract_data()
    # Do the rest of the ETL process here (transform and load)
    # ...
    logging.info("ETL Job Completed Successfully")

# Run the ETL process
run_etl()

# Extract

Question 2 Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:

In [24]:
# Call the function here
data = extract_data()
# Print the rows here
print(data.head())

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399


 # Transform
Log the following  <code>"Transform phase Started"</code>

In [53]:
transformed_data = transform_data(data, exchange_rate)

In [54]:
# Call the function here

# Print the first 5 rows here
print(transformed_data[:5])

[ 8.4 16.8 25.2]


# Question 3 Use the function transform and print the first 5 rows of the output, take a screen shot:

# Log your data "Transform phase Ended"

In [56]:
logging.info("Transform phase Started")

# Call the load function

In [57]:
# Write your code here
def load_data(data, output_file):
    logging.info(f"Loading data to {output_file}")
    # Convert array to DataFrame
    data_frame = pd.DataFrame(data)
    # Save DataFrame to CSV
    data_frame.to_csv(output_file, index=False)

In [58]:
load_data(transformed_data, 'output.csv')

# Log the following "Load phase Ended".

In [59]:
logging.info("Load phase Ended")